In [1]:
# %pip install --upgrade --quiet  rank_bm25 langchain-voyageai

Please prepare you [VOYAGE_API_KEY](https://docs.voyageai.com/docs/api-key-and-installation#authentication-with-api-keys) in your environment variables.

In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from rag_utils.vanilla import vectorstore

# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()
# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

In [3]:
vectorstore.add_documents(docs)
milvus_retriever = vectorstore.as_retriever()

In [4]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(docs)

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.vanilla import format_docs, rag_prompt, llm


vanilla_rag_chain = (
    {"context": milvus_retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [6]:
from langchain_voyageai import VoyageAIRerank
from rag_utils.hybrid_and_rerank import RerankerRunnable

reranker = RerankerRunnable(
    compressor=VoyageAIRerank(model="rerank-lite-1"),
    top_k=4,
)

hybrid_and_rerank_retriever = {
    "milvus_retrieved_doc": milvus_retriever,
    "mb25_retrieved_doc": bm25_retriever,
    "query": RunnablePassthrough(),
} | reranker

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

hybrid_and_rerank_chain = (
    {
        "context": hybrid_and_rerank_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [10]:
query = "Which model use tools?"

vanilla_result = vanilla_rag_chain.invoke(query)
hybrid_and_rerank_result = hybrid_and_rerank_chain.invoke(query)
print(
    f"\n[vanilla_result]:\n{vanilla_result}\n\n[hybrid_and_rerank_result]:\n{hybrid_and_rerank_result}"
)

len(milvus_retrieved_doc) = 4
len(mb25_retrieved_doc) = 4
len(unique_documents) = 6

[vanilla_result]:
The models that use tools are TALM (Tool Augmented Language Models; Parisi et al. 2022), Toolformer (Schick et al. 2023), and MRKL (Modular Reasoning, Knowledge and Language; Karpas et al. 2022). These models are fine-tuned to learn to use external tool APIs and other expert modules for tasks such as code execution, accessing proprietary information sources, and more.

[hybrid_and_rerank_result]:
The models that use tools are ChatGPT with Plugins and OpenAI API function calling, HuggingGPT, TALM (Tool Augmented Language Models), Toolformer, and MRKL (Modular Reasoning, Knowledge and Language). These models are equipped with the capability to use external tools or APIs to extend their functionalities.


In [hybrid_and_rerank_result], it answers with more ground truth:
> ChatGPT with Plugins and OpenAI API function calling, HuggingGPT

which does not appear in [vanilla result].